<h1><center>Farrukh's Week 3 - Assignment Part 3</center></h1>


In [91]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

## Visulaziations
from geopy.geocoders import Nominatim
import folium
import time

## KMeans
from sklearn.cluster import KMeans

from matplotlib import pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline
print("Loaded all required Librarires!!!")

Loaded all required Librarires!!!


#### Load the Pandas DataFrame We Stored Earlier

In [92]:
toronto_neighborhoods = pd.read_csv('toronto_data.csv', index_col=0)

Let us describe the data frame

In [93]:
toronto_neighborhoods.describe(include='all')

,PostalCode,Unnamed: 0.1,Neighborhood,Borough,Latitude,Longitude
count,103,103.000000,103,103,103.000000,103.000000
unique,103,NaN,103,11,NaN,NaN
top,M6M,NaN,Rosedale,North York,NaN,NaN
freq,1,NaN,1,24,NaN,NaN
mean,NaN,78.087379,NaN,NaN,43.704608,-79.397153
std,NaN,49.346734,NaN,NaN,0.052463,0.097146
min,NaN,1.000000,NaN,NaN,43.602414,-79.615819
25%,NaN,40.500000,NaN,NaN,43.660567,-79.464763
50%,NaN,77.000000,NaN,NaN,43.696948,-79.388790
75%,NaN,105.500000,NaN,NaN,43.745320,-79.340923


### Exploring Visualizaitons and Analysis

Finding out how many Boroughs have unique "Postal Codes" (103)

In [94]:
print(toronto_neighborhoods['Borough'].value_counts())

North York          24
Downtown Toronto    18
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East York            5
East Toronto         5
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64


In [95]:
print('Unique Boroughs:', toronto_neighborhoods['Borough'].value_counts().shape[0])

Unique Boroughs: 11


In [96]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="my_explorer")
location = geolocator.geocode(address)
toronto_lat = location.latitude
toronto_lon = location.longitude
print('The geograpical coordinate of Toronto in Canada are {}, {}.'.format(toronto_lat, toronto_lon))

The geograpical coordinate of Toronto in Canada are 43.653963, -79.387207.


#### Map of Toronto with Postal codes on Top

In [97]:
toronto_map = folium.Map(location=[toronto_lat, toronto_lon], zoom_start=10)

# add all postcoe markers on te map
for lat, lng, borough, neighborhood in zip(toronto_neighborhoods['Latitude'], toronto_neighborhoods['Longitude'], toronto_neighborhoods['Borough'], toronto_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(toronto_neighborhoods, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

Let us find out neighborhoods from **North York**

In [98]:
neighborhoods = toronto_neighborhoods[toronto_neighborhoods['Borough']=='North York']

In [99]:
map_toronto_north_york = folium.Map(location=[toronto_lat, toronto_lon], zoom_start=12)

# add all postcoe markers on te map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_north_york)  
    
map_toronto_north_york

# Let's not get data for all locations from Foursquare

In [100]:
CLIENT_ID = 'OISUGEKUBDCCKKGXLPZ4DLJX4BLO1NUU2RS43M13NQUFUOZH' # your Foursquare ID
CLIENT_SECRET = 'ZLNH2R045C4XWP2PTTOZNET4QZ41X1CJ54Y1QPHJ5UFHDNZK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OISUGEKUBDCCKKGXLPZ4DLJX4BLO1NUU2RS43M13NQUFUOZH
CLIENT_SECRET:ZLNH2R045C4XWP2PTTOZNET4QZ41X1CJ54Y1QPHJ5UFHDNZK


In [101]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Anaylzing neighbourhoods 

In [102]:
def getNearbyVenues(names, latitudes, longitudes, limit=100, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return nearby_venues

In [103]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )
print(toronto_venues.shape)
toronto_venues.head()

Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Bedford Park,Lawrence Manor East
Lawrence Heights,Lawrence Manor
Glencairn
Downsview,North Park,Upwood Park
Humber Summit
Emery,Humberlea
(241, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
2,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
3,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run
4,"Fairview,Henry Farm,Oriole",43.778517,-79.346556,The LEGO Store,43.778207,-79.343483,Toy / Game Store


In [104]:
#Saving our data frame
toronto_venues.to_csv('toronto_venues.csv')

Let's check how many venues were returned for each neighborhood

In [105]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor,Downsview North,Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",24,24,24,24,24,24
"CFB Toronto,Downsview East",2,2,2,2,2,2
Don Mills North,4,4,4,4,4,4
Downsview Central,4,4,4,4,4,4
Downsview Northwest,5,5,5,5,5,5
Downsview West,6,6,6,6,6,6
"Downsview,North Park,Upwood Park",4,4,4,4,4,4


Also let's check how manu diffferent categoties do we have here for all the venues and what they are

In [106]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 104 uniques categories.


In [107]:
toronto_venues['Venue Category'].value_counts()

Coffee Shop                   18
Clothing Store                12
Fast Food Restaurant           9
Pizza Place                    8
Japanese Restaurant            8
Grocery Store                  7
Park                           6
Shopping Mall                  5
Bank                           5
Sushi Restaurant               5
Restaurant                     5
Sandwich Place                 5
Café                           5
Furniture / Home Store         4
Liquor Store                   4
Discount Store                 4
Asian Restaurant               4
Juice Bar                      4
Pharmacy                       4
Women's Store                  3
Electronics Store              3
Home Service                   3
Ramen Restaurant               3
Baseball Field                 3
Caribbean Restaurant           3
Bakery                         3
Italian Restaurant             3
Deli / Bodega                  2
Pub                            2
Gym                            2
          

#### Let us 'digitize' description of each neighborhood

In [108]:
# one hot enc
df_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [df_onehot.columns[-1]] + list(df_onehot.columns[:-1])
df_onehot = df_onehot[fixed_columns]
df_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Fairview,Henry Farm,Oriole",0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


# Let's take a closer look at TOP-4 most common venues in each neighborhood we analyze

because for one of the neighborhoods we got only 4 venues in Toronto Downtown

Also, let's sort them in descending order.

In [109]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


toronto_downtown_grouped = df_onehot.groupby('Neighborhood').mean().reset_index()

num_top_venues = 4

#Because we will look only at 4-top most common venues let's simpliy the columns naming
columns = ['1st Most Common Venue', '2nd Most Common Venue', '3rd Most Common Venue', '4th Most Common Venue']

# create columns according to number of top venues
columns = ['Neighborhood'] + columns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_downtown_grouped['Neighborhood']

for ind in np.arange(toronto_downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
0,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Bank,Bridal Shop,Diner
1,Bayview Village,Japanese Restaurant,Chinese Restaurant,Café,Bank
2,"Bedford Park,Lawrence Manor East",Juice Bar,Italian Restaurant,Sushi Restaurant,Coffee Shop
3,"CFB Toronto,Downsview East",Park,Airport,Dog Run,Comfort Food Restaurant
4,Don Mills North,Japanese Restaurant,Caribbean Restaurant,Gym / Fitness Center,Café
5,Downsview Central,Food Truck,Home Service,Baseball Field,Korean Restaurant
6,Downsview Northwest,Grocery Store,Gym / Fitness Center,Athletics & Sports,Liquor Store
7,Downsview West,Grocery Store,Bank,Hotel,Park
8,"Downsview,North Park,Upwood Park",Park,Construction & Landscaping,Bakery,Basketball Court
9,"Emery,Humberlea",Baseball Field,Women's Store,Electronics Store,Concert Hall


### KMeans Cluster Neighborhoods

We will potentially uncover 3 clusters Some what like: Food places, Playgrounds and Airport. 

In [110]:
# set number of clusters
kclusters = 3

grouped_clustering = toronto_downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=int32)

 Let's create a new dataframe that includes the cluster as well as the top 4 venues for each neighborhood.

In [111]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# merge toronto_grouped with toronto  neiborhoods data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Unnamed: 0.1,Neighborhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
17,M2H,25,Hillcrest Village,North York,43.803762,-79.363452,0.0,Golf Course,Mediterranean Restaurant,Pool,Dog Run
18,M2J,26,"Fairview,Henry Farm,Oriole",North York,43.778517,-79.346556,0.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Women's Store
19,M2K,27,Bayview Village,North York,43.786947,-79.385975,0.0,Japanese Restaurant,Chinese Restaurant,Café,Bank
20,M2L,28,"Silver Hills,York Mills",North York,43.757490,-79.374714,2.0,Cafeteria,Women's Store,Coffee Shop,Concert Hall
21,M2M,29,"Newtonbrook,Willowdale",North York,43.789053,-79.408493,NaN,NaN,NaN,NaN,NaN


It is interesting to see how many memebers each detected cluster has and what they are

In [112]:
toronto_merged['Cluster Labels'].value_counts()

0.0    21
1.0     1
2.0     1
Name: Cluster Labels, dtype: int64

Seemslike it is working out

In [113]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2]

,PostalCode,Unnamed: 0.1,Neighborhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
20,M2L,28,"Silver Hills,York Mills",North York,43.75749,-79.374714,2.0,Cafeteria,Women's Store,Coffee Shop,Concert Hall


In [114]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]

,PostalCode,Unnamed: 0.1,Neighborhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
97,M9M,169,"Emery,Humberlea",North York,43.724766,-79.532242,1.0,Baseball Field,Women's Store,Electronics Store,Concert Hall


Our clustering makes sence  - as expected we got food places, airport and playground in three separate clusters. 

# Visulize Cluster

In [115]:
import math

# create map
map_clusters = folium.Map(location=[toronto_lat, toronto_lon], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], 
                                  toronto_merged['Longitude'], 
                                  toronto_merged['Neighborhood'], 
                                  toronto_merged['Cluster Labels']):
    if not math.isnan(cluster):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[int(cluster)-1],
            fill=True,
            fill_color=rainbow[int(cluster)-1],
            fill_opacity=0.7).add_to(map_clusters)
       
map_clusters
    